In [1]:
import pandas as pd
import os

Creating the Dataframes of Camera Data

In [15]:
#Set directories for csvs of pose and activity data
actDir = "act_csvs"
camDir = "cam_csvs"
#set the range in seconds where to label frames as interacting
range = 30

In [6]:
#write all filenames of pose csvs into an array
filenames = []
directory = os.fsencode(camDir)
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".csv"):
         filenames.append(filename)

print(filenames)
print(len(filenames))

['2023-03-01T06-35-11.162.csv', '2023-03-02T07-51-54.687.csv', '2023-03-01T10-16-45.731.csv', '2023-03-23T10-18-12.982.csv', '2023-03-23T13-22-57.733.csv', '2023-03-23T20-42-27.709.csv', '2023-03-02T08-42-40.687.csv', '2023-03-02T07-03-35.233.csv', '2023-03-01T09-11-44.103.csv', '2023-03-01T03-41-08.025.csv', '2023-03-02T00-36-18.194.csv', '2023-03-01T15-27-27.603.csv', '2023-03-02T19-27-26.668.csv', '2023-03-01T08-46-48.651.csv', '2023-03-01T13-56-56.858.csv', '2023-03-01T16-37-40.233.csv', '2023-03-01T10-16-02.876.csv', '2023-03-23T14-57-43.268.csv', '2023-03-02T12-13-38.972.csv', '2023-03-01T13-40-50.182.csv', '2023-03-23T16-08-00.885.csv', '2023-03-01T08-23-49.356.csv', '2023-03-23T10-49-02.070.csv', '2023-03-01T11-53-45.725.csv', '2023-03-01T19-39-16.147.csv', '2023-03-01T11-46-03.941.csv', '2023-03-02T08-08-40.886.csv', '2023-03-23T10-49-18.121.csv', '2023-03-23T11-34-59.743.csv', '2023-03-02T07-34-31.722.csv', '2023-03-02T08-14-10.702.csv', '2023-03-23T09-42-40.919.csv', '2023-0

In [11]:
#extract the pose csv data into an array of dataframes
datasets = []
countempty = 0
for file in filenames:
    filestring = "./" + camDir + "/" + file
    df = pd.read_csv(filestring)
    if not df.empty:
        datasets.append(pd.read_csv(filestring))
    else:
        countempty += 1

print('dataframes: ' + str(len(datasets)))
print('empty: ' + str(countempty))
datasets[0].head()

dataframes: 665
empty: 663


,time,gro1,gro2,gro3,gro4,v1,v2,v3,k0x,k0y,...,k30z,k31x,k31y,k31z,k32x,k32y,k32z,k33x,k33y,k33z
0,"2023, 3, 1, 6, 35, 11, 162000",0.01090,0.10627,0.00737,0.99425,0.00000,0.00000,0.00000,-2.35441,0.57753,...,7.67831,-2.28680,-0.12841,7.58714,-2.41627,1.42584,7.69294,-2.29491,1.42900,7.64643
1,"2023, 3, 1, 6, 35, 11, 295000",0.01715,-0.00961,-0.00830,0.99977,-1.33345,0.11505,3.15966,-2.51104,0.58596,...,8.05493,-2.44572,-0.11720,7.97229,-2.56547,1.45320,8.03477,-2.41340,1.45175,8.03889
2,"2023, 3, 1, 6, 35, 11, 395000",0.02417,-0.12159,0.01269,0.99221,-1.75139,0.17629,5.03528,-2.74246,0.60646,...,8.70303,-2.63232,-0.10079,8.63328,-2.86464,1.46947,8.80713,-2.72503,1.46148,8.69365
3,"2023, 3, 1, 6, 35, 11, 478000",0.01247,-0.17533,0.02139,0.98420,-1.71678,0.17560,4.98434,-2.82673,0.60875,...,8.93043,-2.69798,-0.10615,8.86750,-2.98836,1.47901,9.00140,-2.72571,1.37535,8.67796
4,"2023, 3, 1, 6, 35, 11, 578000",0.00357,-0.16166,0.03296,0.98629,-1.56744,0.15738,4.49958,-2.90764,0.61333,...,9.10533,-2.76641,-0.10782,9.03932,-3.10449,1.50480,9.12168,-2.88318,1.48190,8.93440


Creating the Ground Truth and preparing Machine Learning

In [23]:
truths = []
ones = 0
zeros = 0
counter = 0
#iterate through all dataframes
for data in datasets:
    #extract the first column which contains the time
    dates = data.loc[:,"time"]
    #if date is unreadable, set it to an impossible date
    try:
        date = dates[0].replace(" ", "").split(',')[0:3]
        year = date[0]
        month = date[1]
        if len(month) < 2:
            month = '0' + month
        day = date[2]
        if len(day) < 2:
            day = '0' + day
    except:
        year = 0
        month = 0
        day = 0
        pass

    #create the string of the associated activity csv from the extracted time
    filestring = './' + actDir + '/activities-' + year + '-' + month + '-' + day + '.csv'

    labels = []
    #open the corresponding file if it exists
    try:
        timestamps = pd.read_csv(filestring)
    except:
        pass

    #iterate through all dates in the time column of the dataframe
    for date in dates:
        #extract the time of day in seconds of a given frame
        try:
            time = date.replace(" ", "").split(',')[3:6]
            hour = int(time[0])
            minute = int(time[1])
            second = int(time[2])
            total = hour * 3600 + minute * 60 + second
        except:
            #if time is unreadable, set it outside of possible range
            counter += 1
            total = 100000
            pass

        #set the label to 1 if the time is within 20 seconds of an activity, else leave it at 0
        found = 0
        for timestamp in timestamps:
            if found == 0:
                ts = timestamp.split('T')[1].split('.')[0]
                ts_h = int(ts.split(':')[0])
                ts_m = int(ts.split(':')[1])
                ts_s = int(ts.split(':')[2])
                ts_total = ts_h * 3600 + ts_m * 60 + ts_s

                if ts_total + range > total > ts_total - range:
                    found = 1
                    labels.append(1)
                    ones += 1

        if found == 0:
            labels.append(0)
            zeros += 1

    #create a list of lists that contains the labels to each dataframe
    truths.append(labels)

print(truths)
print('nr of 0s: ' + str(zeros))
print('nr of 1s: ' + str(ones))
print('nr of unreadable dates (set to 0): ' + str(counter))

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [24]:
#concatenate all dataframes to one single dataframe
megatable = datasets[0]
for data in datasets[1:]:
    megatable = pd.concat([megatable, data], ignore_index=True)

#concatenate all the label lists to one big list that corresponds row for row with the dataframe
y = []
for truth in truths:
    y += truth

print(megatable)
print(y)

                                   time     gro1     gro2     gro3     gro4  \
0         2023, 3, 1, 6, 35, 11, 162000   0.0109  0.10627  0.00737  0.99425   
1         2023, 3, 1, 6, 35, 11, 295000  0.01715 -0.00961  -0.0083  0.99977   
2         2023, 3, 1, 6, 35, 11, 395000  0.02417 -0.12159  0.01269  0.99221   
3         2023, 3, 1, 6, 35, 11, 478000  0.01247 -0.17533  0.02139   0.9842   
4         2023, 3, 1, 6, 35, 11, 578000  0.00357 -0.16166  0.03296  0.98629   
...                                 ...      ...      ...      ...      ...   
156089    2023, 3, 23, 12, 8, 46, 64000 -0.04767  0.98183  0.03316  0.18067   
156090   2023, 3, 23, 12, 8, 46, 164000 -0.03271  0.98156  0.04134  0.18374   
156091   2023, 3, 23, 12, 8, 46, 297000 -0.01415  0.98919   0.0334  0.14208   
156092   2023, 3, 23, 12, 8, 46, 381000 -0.00264  0.99367  0.02975  0.10833   
156093   2023, 3, 23, 12, 8, 46, 481000  0.02044  0.99867  0.04737  0.00211   

             v1       v2       v3      k0x      k0y

Machine Learning